In [5]:
import numpy as np
import rasterio
from rasterio.windows import Window
import matplotlib.pyplot as plt
import os

def aggregate_raster(input_path, output_path, agg_factor, agg_func=np.sum):
    # Open the source raster
    with rasterio.open(input_path) as src:
        # Read the source raster data
        data = src.read(1)
        # Get the transform and metadata
        transform = src.transform
        meta = src.meta.copy()
        
        # Define the new shape after aggregation
        new_height = data.shape[0] // agg_factor
        new_width = data.shape[1] // agg_factor
        
        # Initialize an array to hold the aggregated data
        agg_data = np.zeros((new_height, new_width), dtype=data.dtype)
        
        # Perform the aggregation
        for i in range(new_height):
            for j in range(new_width):
                # Define the window
                window = data[i*agg_factor:(i+1)*agg_factor, j*agg_factor:(j+1)*agg_factor]
                # Apply the aggregation function
                agg_data[i, j] = agg_func(window)
        
        # Update the transform for the new raster
        new_transform = transform * transform.scale((src.width / agg_data.shape[1]), (src.height / agg_data.shape[0]))
        
        # Update the metadata
        meta.update({
            'height': new_height,
            'width': new_width,
            'transform': new_transform
        })
        
        # Write the aggregated data to a new file
        with rasterio.open(output_path, 'w', **meta) as dest:
            dest.write(agg_data, 1)

# Example usage
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'BA_100m') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'BA_1km_vrasterio')
os.makedirs(outfolder, exist_ok=True)

input_raster = os.path.join(outputFolder, 'BA_100m', 'FIREID_2001.tif') 
output_raster = os.path.join(outputFolder, 'BA_1km_vrasterio', 'BA_1km_2001.tif')
aggregation_factor = 10  # Change this to your desired aggregation factor
aggregate_raster(input_raster, output_raster, aggregation_factor)







In [2]:
#import required libraries
import geopandas as gpd
import xarray as xr
import pandas as pd
import os, glob
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.colors
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.cm as cm

import rioxarray as rio
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.windows import Window, transform as rio_window_transform
from rasterio.windows import from_bounds
from rasterio.transform import Affine
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.features import rasterize
from shapely.geometry import box

import dask
import dask.array as da
from dask.delayed import delayed

from osgeo import gdal, ogr, osr

import seaborn as sns
import zipfile

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\4\ipykernel_173084\3466570486.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https

In [23]:
def rescale_and_clip_to_target(input_raster_path, target_raster_path, output_raster_path, compression, dtype):
    """
    Rescale a raster to match the resolution and clip it to the extent of another raster.

    Parameters:
    - input_raster_path: str, path to the input raster file (e.g., 'path/to/input_raster.tif')
    - target_raster_path: str, path to the target raster file (e.g., 'path/to/target_raster.tif')
    - output_raster_path: str, path to the output raster file (e.g., 'path/to/output_raster.tif')
    - compression: str, compression type (e.g., 'zstd', 'deflate', 'lwz')
    - dtype: str, output data type (e.g., 'int16')
    """
    
    with rasterio.open(target_raster_path) as target_src:
        # Get the extent and resolution of the target raster
        target_extent = target_src.bounds
        target_resolution = target_src.res[0]  # Assuming square pixels
    
    with rasterio.open(input_raster_path) as src:
        # Ensure the data type is the same as the target
        if src.dtypes[0] != dtype:
            raise ValueError(f"Input raster is not of type {dtype}")
        
        # Calculate the new transform and dimensions for the target extent
        #new_transform = Affine(target_resolution, 0, target_extent.left, 0, -target_resolution, target_extent.top)
        src_extent = src.bounds
        new_transform = Affine(target_resolution, 0, src_extent.left, 0, -target_resolution, src_extent.top)
        new_width = int((target_extent.right - target_extent.left) / target_resolution)
        new_height = int((target_extent.top - target_extent.bottom) / target_resolution)
        
        # Read the data, resampling and clipping as necessary
        window = from_bounds(*target_extent, transform=src.transform)
        data = src.read(
            out_shape=(src.count, new_height, new_width),
            window=window,
            resampling=Resampling.nearest
        )
        
        # Update metadata
        profile = src.profile
        profile.update({
            'transform': new_transform,
            'width': new_width,
            'height': new_height,
            'res': (target_resolution, target_resolution),
            'dtype': dtype,
            'compress': f'{compression}'
        })
        
        # Write the resampled and clipped data to a new file
        with rasterio.open(output_raster_path, 'w', **profile) as dst:
            dst.write(data.astype(dtype))

In [25]:
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Open the input raster
#input_raster_path = 'input_1km.tif'
input_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
output_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m4.tif'

with rasterio.open(input_raster_path) as src:
    # Read the data and metadata from the input raster
    data = src.read(1)
    transform = src.transform
    profile = src.profile

    # Calculate new transform and dimensions for 100m resolution
    new_transform = rasterio.Affine(transform.a / 10, transform.b, transform.c,
                                    transform.d, transform.e / 10, transform.f)
    new_height = src.height * 10
    new_width = src.width * 10

    # Update the profile with new dimensions and transform
    profile.update({
        'height': new_height,
        'width': new_width,
        'transform': new_transform,
        'compress': 'zstd'
    })

    # Create a new array with the upsampled data
    resampled_data = np.repeat(np.repeat(data, 10, axis=0), 10, axis=1)

    # Write the resampled data to a new file
    with rasterio.open(output_raster_path, 'w', **profile) as dst:
        dst.write(resampled_data, 1)

print(f"Resampled raster saved to {output_raster_path}")


Resampled raster saved to L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m4.tif


In [24]:
# rescale, align EEA refgrid to match fire data extent and resolution
input_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
target_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m3.tif'
compression = 'zstd'
dtype = 'int32'
rescale_and_clip_to_target(input_raster_path, target_raster_path, output_raster_path, compression, dtype)

In [21]:
def rescale_and_clip_to_target2(input_raster_path, target_raster_path, output_raster_path, compression, dtype):
    """
    Rescale a raster to match the resolution and clip it to the extent of another raster.

    Parameters:
    - input_raster_path: str, path to the input raster file (e.g., 'path/to/input_raster.tif')
    - target_raster_path: str, path to the target raster file (e.g., 'path/to/target_raster.tif')
    - output_raster_path: str, path to the output raster file (e.g., 'path/to/output_raster.tif')
    - compression: str, compression type (e.g., 'zstd', 'deflate', 'lwz')
    - dtype: str, output data type (e.g., 'int16')
    """
    
    with rasterio.open(target_raster_path) as target_src:
        # Get the extent and resolution of the target raster
        target_extent = target_src.bounds
        target_resolution = target_src.res[0]  # Assuming square pixels
    
    with rasterio.open(input_raster_path) as src:
        # Ensure the data type is the same as the target
        if src.dtypes[0] != dtype:
            raise ValueError(f"Input raster is not of type {dtype}")
        
        # Calculate the new transform and dimensions for the target extent
        new_transform = Affine(target_resolution, 0, target_extent.left, 0, -target_resolution, target_extent.top)
        new_width = int((target_extent.right - target_extent.left) / target_resolution)
        new_height = int((target_extent.top - target_extent.bottom) / target_resolution)
        
        # Read the data, resampling and clipping as necessary
        window = from_bounds(*target_extent, transform=src.transform)
        data = src.read(
            out_shape=(src.count, new_height, new_width),
            window=window
        )
        
        # Update metadata
        profile = src.profile
        profile.update({
            'transform': new_transform,
            'width': new_width,
            'height': new_height,
            'res': (target_resolution, target_resolution),
            'dtype': dtype,
            'compress': f'{compression}'
        })
        
        # Write the resampled and clipped data to a new file
        with rasterio.open(output_raster_path, 'w', **profile) as dst:
            dst.write(data.astype(dtype))

In [22]:
# rescale, align EEA refgrid to match fire data extent and resolution
input_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
target_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m_tt2.tif'
compression = 'zstd'
dtype = 'int32'
rescale_and_clip_to_target2(input_raster_path, target_raster_path, output_raster_path, compression, dtype)

In [ ]:
import rasterio
import numpy as np

def load_raster(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
    return data

def compute_pixel_counts(zone_raster, value_raster):
    zones = np.unique(zone_raster)
    counts = {}
    
    for zone in zones:
        if zone == 0:
            continue  # Skip background or no-data zone if present
        mask = (zone_raster == zone) & (value_raster != 0)  # Assuming non-zero values are valid
        counts[zone] = np.sum(mask)
        
    return counts

# Load rasters
zone_raster = load_raster('rescaled_zone_raster.tif')
value_raster = load_raster('aligned_value_raster.tif')

# Compute pixel counts
pixel_counts = compute_pixel_counts(zone_raster, value_raster)
print(pixel_counts)



In [26]:
zone_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
value_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')

In [3]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np


# Paths to input and output files
value_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
extent_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = os.path.join(outputFolder, 'devdata', 'grid1KmLAND_v2_enlarged.tif')

# Open the value raster
with rasterio.open(value_raster_path) as value_raster:
    value_data = value_raster.read(1)
    value_transform = value_raster.transform
    value_crs = value_raster.crs
    value_nodata = value_raster.nodata

# Open the extent raster
with rasterio.open(extent_raster_path) as extent_raster:
    extent_transform = extent_raster.transform
    extent_crs = extent_raster.crs
    extent_width = extent_raster.width
    extent_height = extent_raster.height

# Initialize the output array with nodata values
output_data = np.full((extent_height, extent_width), value_nodata, dtype=value_data.dtype)

# Calculate the coordinates of the extent raster
extent_bounds = extent_raster.bounds
extent_left, extent_bottom = extent_bounds.left, extent_bounds.bottom

# Calculate the window of the value raster that fits within the extent raster
value_window = rasterio.windows.from_bounds(
    left=extent_left, bottom=extent_bottom,
    right=extent_bounds.right, top=extent_bounds.top,
    transform=value_transform
)

# Reproject the value raster to match the extent raster
reproject(
    source=value_data,
    destination=output_data,
    src_transform=value_transform,
    src_crs=value_crs,
    dst_transform=extent_transform,
    dst_crs=extent_crs,
    resampling=Resampling.nearest,
    src_nodata=value_nodata,
    dst_nodata=0
)

# Save the output raster
with rasterio.open(
    output_raster_path, 'w',
    driver='GTiff',
    height=extent_height,
    width=extent_width,
    count=1,
    dtype=value_data.dtype,
    crs=extent_crs,
    transform=extent_transform,
    nodata=0,
    compress='zstd'
) as dst:
    dst.write(output_data, 1)


In [1]:
import os
import rasterio
import numpy as np
import dask.array as da
from dask import delayed
from dask.diagnostics import ProgressBar
from rasterstats import zonal_stats
from rasterio.transform import from_origin


# Paths to input and output files
zone_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m5.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
value_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = os.path.join(outputFolder, 'devdata', 'zonalstats_rasterio_tt.tif')

# Load the zone raster and the value raster
with rasterio.open(zone_raster_path) as zone_src:
    zone_array = zone_src.read(1)
    transform = zone_src.transform
    crs = zone_src.crs
    profile = zone_src.profile

with rasterio.open(value_raster_path) as value_src:
    value_array = value_src.read(1)

# Convert the arrays to Dask arrays
zone_dask_array = da.from_array(zone_array, chunks=(1024, 1024))
value_dask_array = da.from_array(value_array, chunks=(1024, 1024))

# Function to compute zonal count for a chunk
def compute_zonal_count(zone_chunk, value_chunk):
    if zone_chunk.size == 0 or value_chunk.size == 0:
        return np.zeros_like(zone_chunk, dtype=np.float32)
    
    stats = zonal_stats(zone_chunk, value_chunk, stats="count")
    zone_stats = {stat['zone']: stat['count'] for stat in stats}
    output_chunk = np.zeros_like(zone_chunk, dtype=np.float32)
    
    for zone_id, count_value in zone_stats.items():
        output_chunk[zone_chunk == zone_id] = count_value
    
    return output_chunk

# Compute the zonal count in parallel
output_dask_array = da.map_blocks(compute_zonal_count, zone_dask_array, value_dask_array, dtype=np.float32)

# Persist the output array in memory
output_dask_array = output_dask_array.persist()

# Write the output raster 
profile.update(dtype=rasterio.float32, count=1, compress='zstd')

with rasterio.open(output_raster_path, 'w', **profile) as dst:
    with ProgressBar():
        for i in range(output_dask_array.shape[0]):
            dst.write(output_dask_array[i].compute(), 1)

print(f"Zonal count raster saved to {output_raster_path}")


ValueError: Specify affine transform for numpy arrays

In [2]:
import rasterio
import numpy as np
import dask.array as da
from dask import delayed
from dask.diagnostics import ProgressBar
from rasterstats import zonal_stats

# Paths to input and output files
zone_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m5.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
value_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = os.path.join(outputFolder, 'devdata', 'zonalstats_rasterio_tt.tif')

# Load the zone raster and the value raster
with rasterio.open(zone_raster_path) as zone_src:
    zone_array = zone_src.read(1)
    transform = zone_src.transform
    crs = zone_src.crs
    profile = zone_src.profile

with rasterio.open(value_raster_path) as value_src:
    value_array = value_src.read(1)

# Check if the shapes are identical
assert zone_array.shape == value_array.shape, "Zone and value rasters must have the same shape"

# Convert the arrays to Dask arrays with identical chunk sizes
chunks = (1024, 1024)
zone_dask_array = da.from_array(zone_array, chunks=chunks)
value_dask_array = da.from_array(value_array, chunks=chunks)

# Function to compute zonal count for a chunk
def compute_zonal_count(zone_chunk, value_chunk):
    if zone_chunk.size == 0 or value_chunk.size == 0:
        return np.zeros_like(zone_chunk, dtype=np.float32)
    
    # Use 'count' statistic to get the number of pixels in each zone
    stats = zonal_stats(zone_chunk, value_chunk, stats="count")
    zone_stats = {stat['zone']: stat['count'] for stat in stats}
    output_chunk = np.zeros_like(zone_chunk, dtype=np.float32)
    
    for zone_id, count_value in zone_stats.items():
        output_chunk[zone_chunk == zone_id] = count_value
    
    return output_chunk

# Compute the zonal count in parallel
output_dask_array = da.map_blocks(compute_zonal_count, zone_dask_array, value_dask_array, dtype=np.float32)

# Persist the output array in memory
output_dask_array = output_dask_array.persist()

# Write the output raster
profile.update(dtype=rasterio.float32, count=1, compress='lzw')

with rasterio.open(output_raster_path, 'w', **profile) as dst:
    with ProgressBar():
        dst.write(output_dask_array.compute(), 1)

print(f"Zonal count raster saved to {output_raster_path}")


ValueError: Specify affine transform for numpy arrays

In [3]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import dask.array as da
from dask import delayed
from dask.diagnostics import ProgressBar
from rasterstats import zonal_stats

# Paths to input and output files
zone_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m5.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
value_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = os.path.join(outputFolder, 'devdata', 'zonalstats_rasterio_tt.tif')

# Load the zone raster
with rasterio.open(zone_raster_path) as zone_src:
    zone_array = zone_src.read(1)
    zone_transform = zone_src.transform
    zone_crs = zone_src.crs
    profile = zone_src.profile

# Load the value raster and resample it to match the zone raster
with rasterio.open(value_raster_path) as value_src:
    value_transform = value_src.transform
    value_crs = value_src.crs
    
    # Calculate the transform and shape for the output raster
    transform, width, height = calculate_default_transform(
        value_crs, zone_crs, value_src.width, value_src.height, *value_src.bounds)
    
    profile.update({
        'crs': zone_crs,
        'transform': zone_transform,
        'width': width,
        'height': height
    })
    
    # Create an array for the resampled data
    value_resampled = np.empty((height, width), dtype=value_src.meta['dtype'])
    
    # Reproject and resample the value raster to match the zone raster
    reproject(
        source=rasterio.band(value_src, 1),
        destination=value_resampled,
        src_transform=value_transform,
        src_crs=value_crs,
        dst_transform=zone_transform,
        dst_crs=zone_crs,
        resampling=Resampling.nearest
    )

# Check if the shapes are identical
assert zone_array.shape == value_resampled.shape, "Zone and value rasters must have the same shape after resampling"

# Convert the arrays to Dask arrays with identical chunk sizes
chunks = (1024, 1024)
zone_dask_array = da.from_array(zone_array, chunks=chunks)
value_dask_array = da.from_array(value_resampled, chunks=chunks)

# Function to compute zonal count for a chunk
def compute_zonal_count(zone_chunk, value_chunk, transform):
    if zone_chunk.size == 0 or value_chunk.size == 0:
        return np.zeros_like(zone_chunk, dtype=np.float32)
    
    # Use 'count' statistic to get the number of pixels in each zone
    stats = zonal_stats(zone_chunk, value_chunk, affine=transform, stats="count")
    zone_stats = {stat['zone']: stat['count'] for stat in stats}
    output_chunk = np.zeros_like(zone_chunk, dtype=np.float32)
    
    for zone_id, count_value in zone_stats.items():
        output_chunk[zone_chunk == zone_id] = count_value
    
    return output_chunk

# Compute the zonal count in parallel
output_dask_array = da.map_blocks(compute_zonal_count, zone_dask_array, value_dask_array, transform=zone_transform, dtype=np.float32)

# Persist the output array in memory
output_dask_array = output_dask_array.persist()

# Write the output raster
profile.update(dtype=rasterio.float32, count=1, compress='lzw')

with rasterio.open(output_raster_path, 'w', **profile) as dst:
    with ProgressBar():
        dst.write(output_dask_array.compute(), 1)

print(f"Zonal count raster saved to {output_raster_path}")


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import rasterio
import numpy as np
from rasterstats import zonal_stats
from rasterio.transform import from_origin
from collections import defaultdict

# Paths to input and output files
zone_raster_path = zone_raster_path
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
value_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = os.path.join(outputFolder, devdata, 'zonalstats_rasterio_tt.tif')

# Load the zone raster and the value raster
with rasterio.open(zone_raster_path) as zone_src:
    zone_array = zone_src.read(1)
    transform = zone_src.transform
    crs = zone_src.crs
    profile = zone_src.profile

with rasterio.open(value_raster_path) as value_src:
    value_array = value_src.read(1)

# Compute zonal statistics
stats = zonal_stats(zone_array, value_array, stats="count")

# Create a dictionary to map zone IDs to their mean values
zone_stats = {stat['zone']: stat['count'] for stat in stats}

# Create an output array to hold the zonal means
output_array = np.zeros_like(zone_array, dtype=np.float32)

# Map the computed mean to the corresponding zones in the output array
for zone_id, mean_value in zone_stats.items():
    output_array[zone_array == zone_id] = mean_value

# Update the profile to reflect the data type of the output array
profile.update(dtype=rasterio.float32, count=1, compress='lzw')

# Write the output array to a new raster file
with rasterio.open(output_raster_path, 'w', **profile) as dst:
    dst.write(output_array, 1)

print(f"Zonal statistics raster saved to {output_raster_path}")


In [ ]:
import dask.array as da
import dask
import rasterio
from rasterstats import zonal_stats

# Open the rasters
with rasterio.open(zone_raster_path) as zone_src:
    zone_array = zone_src.read(1)
    
with rasterio.open(value_raster_path) as value_src:
    value_array = value_src.read(1)

# Convert to Dask arrays
zone_dask_array = da.from_array(zone_array, chunks=(1024, 1024))
value_dask_array = da.from_array(value_array, chunks=(1024, 1024))

# Function to compute zonal statistics for a chunk
def compute_zonal_stats(zone_chunk, value_chunk):
    return zonal_stats(zone_chunk, value_chunk, stats="count mean sum")

# Map the function across the Dask arrays
result = da.map_blocks(compute_zonal_stats, zone_dask_array, value_dask_array, dtype=object).compute()

# Collect the results
zonal_stats_result = [item for sublist in result for item in sublist]

# Print the statistics
for stat in zonal_stats_result:
    print(stat)


In [28]:
import dask.array as da
import rioxarray
import numpy as np
import rasterio
from rasterio.transform import from_origin

def load_raster(file_path):
    data = rioxarray.open_rasterio(file_path, chunks=True).squeeze()
    return data

def compute_pixel_counts_dask(zone_raster):
    unique_zones = np.unique(zone_raster.compute())
    counts = {}
    
    for zone in unique_zones:
        if zone == 0:
            continue  # Skip background or no-data zone if present
        mask = (zone_raster == zone)
        counts[zone] = mask.sum().compute()
        
    return counts

def create_count_raster_dask(zone_raster, counts, profile, output_path):
    count_raster = da.map_blocks(lambda block: np.vectorize(counts.get)(block), zone_raster, dtype=np.uint32)
    count_raster = count_raster.compute()

    profile.update(dtype=rasterio.uint32, count=1)
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(count_raster, 1)



In [ ]:
# Load rasters
zone_raster = load_raster('rescaled_zone_raster.tif')

# Compute pixel counts
counts = compute_pixel_counts_dask(zone_raster)

# Prepare profile
with rasterio.open('rescaled_zone_raster.tif') as src:
    profile = src.profile

# Create and save the count raster
create_count_raster_dask(zone_raster.data, counts, profile, 'output_count_raster.tif')


In [27]:
import dask.array as da
import rioxarray
import numpy as np
import rasterio
from rasterio.transform import from_origin

def load_raster(file_path):
    data = rioxarray.open_rasterio(file_path, chunks=True).squeeze()
    return data

def compute_pixel_counts_dask(zone_raster):
    unique_zones = np.unique(zone_raster.compute())
    counts = {}
    
    for zone in unique_zones:
        if zone == 0:
            continue  # Skip background or no-data zone if present
        mask = (zone_raster == zone)
        counts[zone] = mask.sum().compute()
        
    return counts

def create_count_raster_dask(zone_raster, counts, profile, output_path):
    count_raster = da.map_blocks(lambda block: np.vectorize(counts.get)(block), zone_raster, dtype=np.uint32)
    count_raster = count_raster.compute()

    profile.update(dtype=rasterio.uint32, count=1)
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(count_raster, 1)



In [ ]:
# Load rasters
zone_raster = load_raster(zone_raster_path)

# Compute pixel counts
counts = compute_pixel_counts_dask(zone_raster)

# Prepare profile
with rasterio.open(zone_raster_path) as src:
    profile = src.profile

# Create and save the count raster
create_count_raster_dask(zone_raster.data, counts, profile, 'output_count_raster.tif')


In [20]:
target_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
with rasterio.open(target_raster_path) as target_src:
    # Get the extent and resolution of the target raster
    target_extent = target_src.bounds
    target_resolution = target_src.res[0]  # Assuming square pixels
    new_transform = Affine(target_resolution, 0, target_extent.left, 0, -target_resolution, target_extent.top)
    new_width = int((target_extent.right - target_extent.left) / target_resolution)
    new_height = int((target_extent.top - target_extent.bottom) / target_resolution)
    print(f'width: {new_width} , height: {new_height}')

width: 63727 , height: 54633


In [9]:
target_src.bounds

BoundingBox(left=943900.0, bottom=941700.0, right=7316600.0, top=6405000.0)

In [10]:
target_src.res[0]

100.0

In [12]:
# Calculate the new transform and dimensions for the target extent
new_transform = Affine(target_resolution, 0, target_extent.left, 0, -target_resolution, target_extent.top)
new_width = int((target_extent.right - target_extent.left) / target_resolution)
new_height = int((target_extent.top - target_extent.bottom) / target_resolution)

In [13]:
new_transform

Affine(1000.0, 0.0, 943000.0,
       0.0, -1000.0, 5416000.0)

In [15]:
print(f'width: {new_width} , height: {new_height}')

width: 6374 , height: 4475


In [19]:
input_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
with rasterio.open(input_raster_path) as input_src:
    # Get the extent and resolution of the target raster
    input_src_extent = input_src.bounds
    input_src_resolution = input_src.res[0]  # Assuming square pixels

width = int((input_src_extent.right - input_src_extent.left) / input_src_resolution)
height = int((input_src_extent.top - input_src_extent.bottom) / input_src_resolution)
print(f'width: {width} , height: {height}')

width: 6374 , height: 4475


In [18]:
input_src.bounds

BoundingBox(left=943000.0, bottom=941000.0, right=7317000.0, top=5416000.0)

In [17]:
input_src.res[0]

1000.0

In [ ]:
import rasterio
import numpy as np

def load_raster(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
    return data

def compute_pixel_counts(zone_raster, value_raster):
    zones = np.unique(zone_raster)
    counts = {}
    
    for zone in zones:
        if zone == 0:
            continue  # Skip background or no-data zone if present
        mask = (zone_raster == zone) & (value_raster != 0)  # Assuming non-zero values are valid
        counts[zone] = np.sum(mask)
        
    return counts

# Load rasters
zone_raster = load_raster('rescaled_zone_raster.tif')
value_raster = load_raster('aligned_value_raster.tif')

# Compute pixel counts
pixel_counts = compute_pixel_counts(zone_raster, value_raster)
print(pixel_counts)
